In [1]:
from datetime import datetime

from db_comm import PortfolioDBManager, DATABASE_NAME

In [2]:
pm = PortfolioDBManager(DATABASE_NAME)

Calculation Motor

In [3]:
from portfolio_test.motor import CalculationMotor

In [4]:
cm = CalculationMotor("AAPL")

In [13]:
pm.sell_all_assets()

# withdraw cash
pm.record_transaction(
    tx_type="WITHDRAW", # WITHDRAW or DEPOSIT
    ticker="CASH",
    shares=pm.get_cash_balance(),
    actual_price=1,
    tx_datetime=datetime.now(),
)
pm.record_transaction(
    tx_type="DEPOSIT", # WITHDRAW or DEPOSIT
    ticker="CASH",
    shares=100,
    actual_price=1,
    tx_datetime=datetime.now(),
)

   -> Cash balance adjusted by 610.9345
2025-12-18 23:35:27: ✅ Recorded SELL: AAPL @ 610.9345. Snapshot updated.
2025-12-18 23:35:28: ✅ Recorded WITHDRAW: CASH @ 710.9345. Snapshot updated.
2025-12-18 23:35:28: ✅ Recorded DEPOSIT: CASH @ 100.0. Snapshot updated.


Build calculation motor for AAPL

In [15]:
data = cm.df
data

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits,Adj_Close_ydy
Date,,,,,,,,,
2020-01-02,688.757977,698.895014,686.316756,698.313764,673.954905,135480400,0.0,0.0,NaN
2020-01-03,690.873736,698.848469,689.362500,691.524733,667.402716,146322800,0.0,0.0,673.954905
2020-01-06,683.061770,697.406980,680.643750,697.034972,672.720742,118387200,0.0,0.0,667.402716
2020-01-07,697.127991,699.592486,691.641026,693.756784,669.556789,108872000,0.0,0.0,672.720742
2020-01-08,690.897009,707.823006,690.897009,704.916756,680.327582,132079200,0.0,0.0,669.556789
...,...,...,...,...,...,...,...,...,...
2025-12-11,2595.630057,2600.186966,2546.432977,2585.678989,2585.678989,33248000,0.0,0.0,2592.653989
2025-12-12,2584.469943,2596.746011,2574.426068,2588.003989,2588.003989,39532900,0.0,0.0,2585.678989
2025-12-15,2605.394943,2605.394943,2537.411966,2549.222864,2549.222864,50409100,0.0,0.0,2588.003989


In [16]:
import numpy as np

for date in data.index:
    print(f"- - - - - Date: {date.date()} - - - - -")
    # if price today is lower than yesterday, buy as much as possible
    data['Adj_Close_ydy'] = data['Adj Close'].shift(1)
    if date != data.index.min():
        price = data.loc[date, "Adj Close"]  # price in SEK
        price_yesterday = data.loc[date, "Adj_Close_ydy"]

        if price < price_yesterday:
            #print("Price dropped, buying AAPL")
            cash_balance = pm.get_cash_balance()
            shares_to_buy = np.divide(cash_balance, price)
            if shares_to_buy > 0:
                print("Buying AAPL")
                pm.record_transaction(
                    tx_type="BUY",
                    ticker="AAPL",
                    shares=shares_to_buy,
                    actual_price=price,
                    tx_datetime=date,
                    currency="SEK",
                )
        # if price today is higher than yesterday, sell all shares
        elif price > price_yesterday:
            portfolio = pm.get_portfolio_snapshot()
            if "AAPL" in list(portfolio["ticker"]):
                shares_to_sell = portfolio[portfolio["ticker"] == "AAPL"]["net_shares"].iloc[0]
                if shares_to_sell > 0:
                    pm.record_transaction(
                        tx_type="SELL",
                        ticker="AAPL",
                        shares=shares_to_sell,
                        actual_price=price,
                        tx_datetime=date,
                        currency="SEK",
                    )


- - - - - Date: 2020-01-02 - - - - -
- - - - - Date: 2020-01-03 - - - - -
Buying AAPL
   -> Cash balance adjusted by -100.0
2020-01-03 00:00:00: ✅ Recorded BUY: AAPL @ 100.0. Snapshot updated.
- - - - - Date: 2020-01-06 - - - - -
   -> Cash balance adjusted by 100.7968
2020-01-06 00:00:00: ✅ Recorded SELL: AAPL @ 100.7968. Snapshot updated.
- - - - - Date: 2020-01-07 - - - - -
Buying AAPL
   -> Cash balance adjusted by -100.7968
2020-01-07 00:00:00: ✅ Recorded BUY: AAPL @ 100.7968. Snapshot updated.
- - - - - Date: 2020-01-08 - - - - -
   -> Cash balance adjusted by 102.4183
2020-01-08 00:00:00: ✅ Recorded SELL: AAPL @ 102.4183. Snapshot updated.
- - - - - Date: 2020-01-09 - - - - -
- - - - - Date: 2020-01-10 - - - - -
- - - - - Date: 2020-01-13 - - - - -
- - - - - Date: 2020-01-14 - - - - -
Buying AAPL
   -> Cash balance adjusted by -102.4183
2020-01-14 00:00:00: ✅ Recorded BUY: AAPL @ 102.4183. Snapshot updated.
- - - - - Date: 2020-01-15 - - - - -
- - - - - Date: 2020-01-16 - - - - 

In [17]:
pm.get_portfolio_snapshot()

,ticker,net_shares,last_trade_price,total_position_value
0,AAPL,0.129584,2528.111966,327.6041


In [18]:
# find Price_SEK at max date
last_price = data.loc[data.index.max(), "Adj Close"]
first_price = data.loc[data.index.min(), "Adj Close"]
earnings = pm.get_portfolio_snapshot()["total_position_value"].sum()
holding_gain = last_price/first_price
trading_gain = earnings/100
print(trading_gain)
print(holding_gain)
print(f"Last price: {(holding_gain-1)*100:.2f} %, Earnings: {(trading_gain-1)*100:.2f} %")
if trading_gain < holding_gain:
    print("Overall worse than buy and hold. "
          f"Specifically, just buying would have given you {((holding_gain-1)/(trading_gain-1)):.2f} times better results.")
else:
    print(f"Overall better than buy and hold")
    if holding_gain-1 > 0:
        print(f"{((trading_gain-1)/(holding_gain-1)):.2f} times better than buy and hold")
    elif trading_gain-1 > 0:
        print("Holding was a loss, but trading made a gain")
    else:
        print("Both trading and holding were a loss but trading lost less")

3.276041
3.75115893870661
Last price: 275.12 %, Earnings: 227.60 %
Overall worse than buy and hold. Specifically, just buying would have given you 1.21 times better results.
